# Protein-Ligand example

This loosely follows the technique outlined in [Quantum Computational Quantification of Protein-Ligand Interactions](https://arxiv.org/pdf/2110.08163.pdf). This utilizes a ranking metric.

$E_{b}(\bar{P},W)= E(\bar{P}, W, \bar{L})-E(\bar{W}, \tilde{L})$

Where $E_{b}$ is the binding energy, $\bar{P}$ and $\bar{P}$ is the protein and ligand structure from experiment. $W$ is the explicit water molecules and $\tilde{L}$ is the optimized ligand in a solvent (in this case water). 

## Example: Glucose in hexokinase

The geometry was found at [rcsb](https://www.rcsb.org/structure/1bdg) and is in the file "pdb1bdg.pdb". This contains the protein (hexokinase) with the ligand (glucose) in the pocket along with some explicit solvent of water and sulfate ion. To perform the study we will use RDKIT to separate the ligand and protein+solvent.

In [1]:
try:
    import tangelo
except ImportError:
    %pip install git+https://github.com/goodchemistryco/Tangelo.git --quiet
    %pip install pyscf --quiet
    %pip install rdkit
    %pip install openbabel-wheel

# Download the data folder at https://github.com/goodchemistryco/Tangelo/branches/develop/tangelo/problem_decomposition/tests/incremental/data
import os
if not os.path.isfile("pdb1bdg.pdb"):
    !sudo apt install subversion
    !svn export https://github.com/goodchemistryco/Tangelo/branches/develop/tangelo/problem_decomposition/tests/incremen

In [2]:

from rdkit import Chem
from rdkit.Chem import rdmolops

mol = Chem.MolFromPDBFile("pdb1bdg.pdb")
mol_frags = rdmolops.GetMolFrags(mol, asMols=True)

# Print here for smiles representation of fragments.
#for i, moli in enumerate(mol_frags):
#    print(i, Chem.MolToSmiles(moli))

# 0 and 1 are the protein fragments
protein_mol = rdmolops.CombineMols(mol_frags[0], mol_frags[1])
# 2 is the ligand fragment
ligand_mol = mol_frags[2]

#3 to end is SO4 or H2O molecules.
for i in range(3,len(mol_frags)):
    protein_mol = rdmolops.CombineMols(protein_mol, mol_frags[i])

protein_ligand_combined = rdmolops.CombineMols(protein_mol, ligand_mol)
ligand_mol = Chem.AddHs(ligand_mol, addCoords=1)

# Export Ligand
Chem.MolToXYZFile(ligand_mol, "ligand.xyz")
# Export Protein+Solvent
Chem.MolToPDBFile(protein_mol, "protein.pdb")

## Implicit Solvation for Ligand in water

We will define a new IntegralSolver that implements the ddCOSMO implicit solvation method as outlined in [solvation methods](https://pyscf.org/user/solvent.html). There is only one line changed from IntegralSolverPySCF in compute_mean_field.

In [76]:
from tangelo.toolboxes.molecular_computation.integral_solver_pyscf import IntegralSolverPySCF, mol_to_pyscf
import os
import numpy as np

# Introduce new Integral Solver that includes implicit solvation with ddCOSMO using pyscf
class IntegralSolverPyscf_ddCOSMO(IntegralSolverPySCF):
    def compute_mean_field(self, sqmol):
        """Run a unrestricted/restricted (openshell-)Hartree-Fock calculation and modify/add the following
        variables to sqmol

        Modify sqmol variables.
            sqmol.mf_energy (float): Mean-field energy (RHF or ROHF energy depending on the spin).
            sqmol.mo_energies (list of float): Molecular orbital energies.
            sqmol.mo_occ (list of float): Molecular orbital occupancies (between 0. and 2.).
            sqmol.n_mos (int): Number of molecular orbitals with a given basis set.
            sqmol.n_sos (int): Number of spin-orbitals with a given basis set.

        Add to sqmol:
            self.mo_coeff (ndarray or List[ndarray]): array of molecular orbital coefficients (MO coeffs) if RHF ROHF
                                                        list of arrays [alpha MO coeffs, beta MO coeffs] if UHF

        Args:
            sqmol (SecondQuantizedMolecule): Populated variables of Molecule plus
                sqmol.basis (string): Basis set.
                sqmol.ecp (dict): The effective core potential (ecp) for any atoms in the molecule.
                    e.g. {"C": "crenbl"} use CRENBL ecp for Carbon atoms.
                sqmol.symmetry (bool or str): Whether to use symmetry in RHF or ROHF calculation.
                    Can also specify point group using string. e.g. "Dooh", "D2h", "C2v", ...
                sqmol.uhf (bool): If True, Use UHF instead of RHF or ROHF reference. Default False


        """

        molecule = mol_to_pyscf(sqmol, sqmol.basis, sqmol.symmetry, sqmol.ecp)

        if self.newton:
            sqmol.mean_field = self.scf.RHF(molecule).newton() if not sqmol.uhf else self.scf.UHF(molecule)
        else:
            # This is the line changed to include implicit solvation with ddCOSMO, water is default
            sqmol.mean_field = molecule.RHF().ddCOSMO() if not sqmol.uhf else self.scf.UHF(molecule)
        sqmol.mean_field.verbose = 0

        chkfile_found = False
        if self.chkfile:
            chkfile_found = os.path.exists(self.chkfile)
            sqmol.mean_field.chkfile = self.chkfile

        # Force broken symmetry for uhf calculation when spin is 0 as shown in
        # https://github.com/sunqm/pyscf/blob/master/examples/scf/32-break_spin_symm.py
        if sqmol.uhf and sqmol.spin == 0 and not chkfile_found:
            dm_alpha, dm_beta = sqmol.mean_field.get_init_guess()
            dm_beta[:1, :] = 0
            dm = (dm_alpha, dm_beta)
            sqmol.mean_field.kernel(dm)
        else:
            sqmol.mean_field.init_guess = "chkfile" if chkfile_found else "minao"
            sqmol.mean_field.kernel()

        sqmol.mean_field.analyze()
        if not sqmol.mean_field.converged:
            raise ValueError("Hartree-Fock calculation did not converge")

        if sqmol.symmetry:
            sqmol.mo_symm_ids = list(self.symm.label_orb_symm(sqmol.mean_field.mol, sqmol.mean_field.mol.irrep_id,
                                                              sqmol.mean_field.mol.symm_orb, sqmol.mean_field.mo_coeff))
            irrep_map = {i: s for s, i in zip(molecule.irrep_name, molecule.irrep_id)}
            sqmol.mo_symm_labels = [irrep_map[i] for i in sqmol.mo_symm_ids]
        else:
            sqmol.mo_symm_ids = None
            sqmol.mo_symm_labels = None

        sqmol.mf_energy = sqmol.mean_field.e_tot
        sqmol.mo_energies = sqmol.mean_field.mo_energy
        sqmol.mo_occ = sqmol.mean_field.mo_occ

        sqmol.n_mos = molecule.nao_nr()
        sqmol.n_sos = 2*sqmol.n_mos

        self.mo_coeff = sqmol.mean_field.mo_coeff

    def get_integrals(self, sqmol, mo_coeff=None):
        r"""Computes core constant, one_body, and two-body integrals for all orbitals

        one-body integrals should be in the form
        h[p,q]= \int \phi_p(x)* (T + V_{ext}) \phi_q(x) dx

        two-body integrals should be in the form
        h[p,q,r,s] = \int \phi_p(x) * \phi_q(y) * V_{elec-elec} \phi_r(y) \phi_s(x) dxdy

        Using molecular orbitals \phi_j(x) = \sum_{ij} A_i(x) mo_coeff_{i,j} where A_i(x) are the atomic orbitals.

        For UHF (if sqmol.uhf is True)
        one_body coefficients are [alpha one_body, beta one_body]
        two_body coefficients are [alpha-alpha two_body, alpha-beta two_body, beta-beta two_body]

        where one_body and two_body are appropriately sized arrays for each spin sector.

        Args:
            sqmol (SecondQuantizedMolecule) : SecondQuantizedMolecule populated with all variables defined above
            mo_coeff : Molecular orbital coefficients to use for calculating the integrals, instead of self.mo_coeff

        Returns:
            (float, array or List[array], array or List[array]): (core_constant, one_body coefficients, two_body coefficients)
        """

        # Pyscf molecule to get integrals.
        pyscf_mol = mol_to_pyscf(sqmol, sqmol.basis, sqmol.symmetry, sqmol.ecp)
        if mo_coeff is None:
            mo_coeff = self.mo_coeff

        if sqmol.uhf:
            one_body, two_body = self.compute_uhf_integrals(sqmol, mo_coeff)
            return float(sqmol.mean_field.energy_nuc()), one_body, two_body

        # Corresponding to nuclear repulsion energy and static coulomb energy.
        core_constant = float(sqmol.mean_field.energy_nuc())

        # get_hcore is equivalent to int1e_kin + int1e_nuc + solvent.v.
        dm1 = sqmol.mean_field.make_rdm1(sqmol.mo_coeff, sqmol.mean_field.mo_occ)
        one_electron_integrals = mo_coeff.T @ (sqmol.mean_field.get_hcore()+sqmol.mean_field.with_solvent.v) @ mo_coeff

        # Add solvent.e and remove duplication of Tr(solvent.v, dm1)
        core_constant += sqmol.mean_field.with_solvent.e - np.einsum('ij,ji->', sqmol.mean_field.with_solvent.v, dm1)

        # Getting 2-body integrals in atomic and converting to molecular basis.
        two_electron_integrals = self.ao2mo.kernel(sqmol.mean_field.mol.intor("int2e"), mo_coeff)
        two_electron_integrals = self.ao2mo.restore(1, two_electron_integrals, len(mo_coeff))

        # PQRS convention in openfermion:
        # h[p,q]=\int \phi_p(x)* (T + V_{ext}) \phi_q(x) dx
        # h[p,q,r,s]=\int \phi_p(x)* \phi_q(y)* V_{elec-elec} \phi_r(y) \phi_s(x) dxdy
        # The convention is not the same with PySCF integrals. So, a change is
        # made before performing the truncation for frozen orbitals.
        two_electron_integrals = two_electron_integrals.transpose(0, 2, 3, 1)

        return core_constant, one_electron_integrals, two_electron_integrals

In [77]:
from tangelo.toolboxes.molecular_computation.molecule import atom_string_to_list
with open("ligand.xyz") as file:
    data = file.read()
ligand_list = atom_string_to_list(data)

In [78]:
from tangelo import SecondQuantizedMolecule
ligand_sq_mol = SecondQuantizedMolecule(ligand_list, 0, 0, IntegralSolverPyscf_ddCOSMO(), "sto-3g")


In [79]:
print(ligand_sq_mol.mf_energy)

-674.4533978096994


In [80]:
#dm = ligand_sq_mol.mean_field.make_rdm1(ligand_sq_mol.mo_coeff, ligand_sq_mol.mo_occ)
#ligand_sq_mol.mean_field.with_solvent.kernel(dm)
print(ligand_sq_mol.mean_field.with_solvent)

In [94]:
from tangelo.toolboxes.molecular_computation.frozen_orbitals import get_orbitals_excluding_homo_lumo
excluding_homo_lumo = get_orbitals_excluding_homo_lumo(ligand_sq_mol)
ligand_sq_mol.freeze_mos(excluding_homo_lumo)

In [92]:
from tangelo.algorithms.variational import VQESolver, BuiltInAnsatze

vqe = VQESolver({"molecule": ligand_sq_mol, "ansatz": BuiltInAnsatze.UCCSD})
vqe.build()



In [93]:
energy_vqe = vqe.simulate()
print(energy_vqe)

-674.453633707917


## QMMM with partial charges for Protein environment

In the paper they state that "Protein structures were aligned with respect to the 1b protein-ligand complex. Fixed-point charges were placed on the protein atoms using the AMBER10:EHT method, as implemented in MOE." 

AMBER10 is not supported in RDKIT which is natively supported in Tangelo so we will use partial charges of the protein+solvent enviroment using the MMFF94 force field. Below we define a version of this that solves for the charges after adding Hs to the protein. The default version in Tangelo does not do this. One can implement your own interface that includes AMBER10:EHT by implementing the get_charges function as defined in the parent `MMChargesSolver` class.

In [101]:
from typing import List, Tuple
from tangelo.toolboxes.molecular_computation.mm_charges_solver import convert_files_to_pdbs, MMChargesSolver

class MMChargesSolverRDKit(MMChargesSolver):
    def __init__(self, mmffVariant="MMFF94"):
        self.mmffVariant = mmffVariant

    def get_charges(self, files) -> List[Tuple[float, Tuple[float, float, float]]]:
        import rdkit.Chem as rdkc
        from rdkit.Chem import AllChem

        pdbfiles = convert_files_to_pdbs(files)
        rdmol = rdkc.rdmolfiles.MolFromPDBFile(pdbfiles[0], removeHs=False)
        rdmol = rdkc.AddHs(rdmol, addCoords=1)
        for file in pdbfiles[1:]:
            mol_to_add = rdkc.rdmolfiles.MolFromPDBFile(file, removeHs=False)
            rdmol = rdkc.rdmolops.CombineMols(rdmol, mol_to_add)
        rdkc.SanitizeMol(rdmol)

        # Read charges and geometry
        mmff_props = AllChem.MMFFGetMoleculeProperties(rdmol, mmffVariant=self.mmffVariant)
        charges = [mmff_props.GetMMFFPartialCharge(i) for i in range(rdmol.GetNumAtoms())]
        rdkc.SanitizeMol(rdmol)
        new_xyz = rdkc.rdmolfiles.MolToXYZBlock(rdmol)

        # Strip first two lines and convert to standard format
        geom = atom_string_to_list("".join([val+"\n" for val in new_xyz.split('\n')[2:]]))

        return charges, geom

We are going to once again only use the HOMO+LUMO for the post- Hartree-Fock calculation with VQE and the QCC ansatz.

In [100]:
from tangelo.problem_decomposition.qmmm.qmmm_problem_decomposition import QMMMProblemDecomposition
from tangelo.problem_decomposition.oniom._helpers.helper_classes import Fragment, Link

# Test when supplying an integral solver.
frag = Fragment(solver_high="vqe", broken_links=[],
                        options_high={"basis": "sto-3g", "ansatz": BuiltInAnsatze.QCC,
                                      "up_then_down": True, "frozen_orbitals": excluding_homo_lumo})
qmmm = QMMMProblemDecomposition({"geometry": ligand_list, "charges": ["protein.pdb"],
                                                "qmfragment": frag, "mmpackage": MMChargesSolverRDKit()})

energy_qmmm = qmmm.simulate()

In [102]:
print(f"The total binding energy is {energy_qmmm-energy_vqe}")


The total binding energy is 0.033990723020110636


We know that glucose binds in hexokinase so this calculation is not accurate enough or makes poor assumptions when the binding energy is positive. 

The paper does add an explicit water solvation shell to the ligand-in-protein calculation which may be straightforward to implement (by modifying `MMChargesSolverRDKit`) and could improve the results.